In [3]:
import os
import pandas as pd
import numpy as np

# ==============================
# 0. 경로 및 기본 설정
# ==============================

# 방금 만든 prefix+feature CSV (경로만 본인 환경에 맞게 수정)
INPUT_PPM_PATH = "./../cohort/cohort_102_recalc_cum.csv"

# ver142에서 이미 만들어 둔 event_id_map
EVENT_ID_MAP_PATH = "./../cohort/cohort_ver142_event_id_map.csv"

OUTPUT_DIR = "./../cohort"
os.makedirs(OUTPUT_DIR, exist_ok=True)

# event 이름/시퀀스까지 붙인 최종 출력 경로
OUTPUT_PPM_WITH_EVENTS_PATH = os.path.join(
    OUTPUT_DIR,
    "cohort_ver144_ppm_with_event_names.csv"
)


# ==============================
# 1. 공통 유틸
# ==============================

def _check_columns(df: pd.DataFrame, required_cols):
    missing = [c for c in required_cols if c not in df.columns]
    if missing:
        raise ValueError(f"필수 컬럼 {missing} 이(가) 누락되어 있습니다. 현재 컬럼: {list(df.columns)}")


# ==============================
# 2. PPM prefix 데이터 로딩
# ==============================

def load_ppm_dataset(path: str) -> pd.DataFrame:
    """
    방금 만든 prefix 기반 cohort CSV 로딩.
    필수 컬럼(현재 102 버전 기준):
      - subject_id
      - hadm_id
      - prefix_len
      - current_event_id
      - next_event_id 또는 target_next_evt
      - time_since_start_min (있으면 사용)
      - target_time_to_next (있으면 time_to_next_min으로 복사)
    """
    if not os.path.exists(path):
        raise FileNotFoundError(f"입력 PPM 데이터 파일을 찾을 수 없습니다: {path}")

    df = pd.read_csv(path)

    # next_event_id 이름이 다를 수 있으므로 보정
    if "next_event_id" not in df.columns and "target_next_evt" in df.columns:
        df = df.rename(columns={"target_next_evt": "next_event_id"})

    # 이 스크립트에서 진짜로 필요한 최소 컬럼만 체크
    required_cols = [
        "subject_id",
        "hadm_id",
        "prefix_len",
        "current_event_id",
        "next_event_id",
    ]
    _check_columns(df, required_cols)

    # time_since_start_min / time_to_next_min 은 있으면 쓰고, 없으면 만들어만 둠
    if "time_since_start_min" not in df.columns:
        df["time_since_start_min"] = np.nan

    # 기존 코드 호환을 위해 time_to_next_min 컬럼 생성
    if "time_to_next_min" not in df.columns:
        if "target_time_to_next" in df.columns:
            df["time_to_next_min"] = df["target_time_to_next"]
        else:
            df["time_to_next_min"] = np.nan

    # prefix 데이터 특성상 prefix_len 순서대로 정렬
    df = df.sort_values(
        by=["subject_id", "hadm_id", "prefix_len"]
    ).reset_index(drop=True)

    print(f"[LOAD] PPM 데이터 로딩 완료: {len(df)} rows, {df['hadm_id'].nunique()} hadm_id")
    print(f"[LOAD] current_event_id 고유 개수: {df['current_event_id'].nunique()}")
    print(f"[LOAD] next_event_id 고유 개수: {df['next_event_id'].nunique()}")

    return df


# ==============================
# 3. event_id_map 로딩
# ==============================

def load_event_id_map(path: str) -> pd.DataFrame:
    """
    ver142에서 만든 event_id_map CSV 로딩.
    필수 컬럼:
      - event_name
      - event_id
    """
    if not os.path.exists(path):
        raise FileNotFoundError(f"event_id_map 파일을 찾을 수 없습니다: {path}")

    m = pd.read_csv(path)
    required_cols = ["event_name", "event_id"]
    _check_columns(m, required_cols)

    # event_id를 int로 보정
    m["event_id"] = m["event_id"].astype(int)

    print(f"[MAP] 이벤트 종류 개수: {len(m)}")
    print(m.sort_values("event_id"))
    return m


# ==============================
# 4. event_id → event_name 매핑 및 prefix 문자열 생성
# ==============================

def attach_event_names_and_prefix(ppm_df: pd.DataFrame,
                                  event_id_map: pd.DataFrame) -> pd.DataFrame:
    """
    - current_event_id / next_event_id → event_name 매핑
    - prefix_len 기준으로 prefix_events_str(이름 기반 시퀀스) 생성
    - full_trace_len 계산
    """
    id_to_name = dict(zip(event_id_map["event_id"], event_id_map["event_name"]))

    # 1) 현재/다음 이벤트 이름 매핑
    ppm_df["current_event"] = ppm_df["current_event_id"].map(id_to_name)
    ppm_df["next_event"] = ppm_df["next_event_id"].map(id_to_name)

    # 매핑 안 된 event_id가 있는지 체크
    unknown_current = ppm_df[ppm_df["current_event"].isna()]["current_event_id"].unique()
    unknown_next = ppm_df[ppm_df["next_event"].isna()]["next_event_id"].unique()

    if len(unknown_current) > 0:
        print(f"[WARN] 매핑되지 않은 current_event_id 존재: {unknown_current}")
    if len(unknown_next) > 0:
        print(f"[WARN] 매핑되지 않은 next_event_id 존재: {unknown_next}")

    # 2) full_trace_len 계산 (각 hadm_id 내에서 최대 prefix_len)
    ppm_df["full_trace_len"] = ppm_df.groupby(
        ["subject_id", "hadm_id"]
    )["prefix_len"].transform("max")

    # 3) prefix_events_str (이름 기반 시퀀스) 생성
    #    각 케이스(subject_id + hadm_id) 별로 prefix_len 순으로 정렬 후
    #    current_event를 누적하여 "ED_ARRIVAL>ECG_TAKEN>..." 형태로 만듦

    def _build_prefix(series: pd.Series) -> pd.Series:
        """
        series: 한 케이스 내 current_event(이름) 시퀀스.
        각 row마다 '여기까지의 이벤트 시퀀스'를 문자열로 반환.
        """
        out = []
        acc = []
        for v in series:
            acc.append(v)
            out.append(">".join(acc))
        return pd.Series(out, index=series.index)

    ppm_df = ppm_df.sort_values(
        ["subject_id", "hadm_id", "prefix_len"]
    )

    # 이미 prefix_events_str가 102에서 있었지만,
    # event_id_map 기준으로 다시 만드는 게 더 일관적이라 덮어씀
    ppm_df["prefix_events_str"] = (
        ppm_df
        .groupby(["subject_id", "hadm_id"])["current_event"]
        .apply(_build_prefix)
        .reset_index(level=[0, 1], drop=True)
    )

    print("\n[BUILD] prefix_events_str 예시:")
    print(
        ppm_df[
            ["subject_id", "hadm_id", "prefix_len",
             "current_event", "next_event", "prefix_events_str"]
        ].head(20)
    )

    return ppm_df


# ==============================
# 5. MAIN
# ==============================

def main():
    # 1) prefix 기반 PPM 데이터 로딩
    ppm_df = load_ppm_dataset(INPUT_PPM_PATH)

    # 2) event_id_map 로딩
    event_id_map = load_event_id_map(EVENT_ID_MAP_PATH)

    # 3) event_name / prefix 문자열 부착
    ppm_with_events = attach_event_names_and_prefix(ppm_df, event_id_map)

    # 4) 저장
    ppm_with_events.to_csv(OUTPUT_PPM_WITH_EVENTS_PATH, index=False)
    print(f"\n[SAVE] 이벤트 이름/시퀀스 포함 PPM 데이터 저장: {OUTPUT_PPM_WITH_EVENTS_PATH}")
    print("[INFO] 작업 완료.")


if __name__ == "__main__":
    main()


[LOAD] PPM 데이터 로딩 완료: 14312 rows, 1929 hadm_id
[LOAD] current_event_id 고유 개수: 8
[LOAD] next_event_id 고유 개수: 9
[MAP] 이벤트 종류 개수: 14
        event_name  event_id
0       ED_ARRIVAL         1
1   ECG_STEMI_FLAG         2
2        ECG_TAKEN         3
3    TROP_POSITIVE         4
4       TROP_TAKEN         5
5   ANTI_PLT_ADMIN         6
6   ANTI_PLT_ORDER         7
7        PCI_START         8
8     ED_DEPARTURE         9
9            DEATH        10
10       DISCHARGE        11
11      ICU_INTIME        12
12     ICU_OUTTIME        13
13             EOS        14
[WARN] 매핑되지 않은 current_event_id 존재: [0]
[WARN] 매핑되지 않은 next_event_id 존재: [0]


TypeError: sequence item 0: expected str instance, float found